In [2]:
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client["detail_film"]
collection =db["imbd_tb"]

In [6]:
cursor = collection.find({},{'Titre original':1,'Duree':1,'_id':0}).sort("Duree", -1).limit(1)

In [7]:
for i in cursor:
    print(i)

{'Titre original': 'Gone with the Wind', 'Duree': 238}


In [8]:
mydoc = collection.find().sort("Duree", -1).limit(1)
for i in mydoc:
    print(i)

{'_id': ObjectId('639b28f15fec497b11d22ca4'), 'Titre original': 'Gone with the Wind', 'Score sur 10': '8.2', 'Genre': 'Drama', 'Date de sorti en USA': '1939', 'Descriptions': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.', 'Duree': 238, 'Acteurs': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell', "Barbara O'Neil", 'Evelyn Keyes', 'Ann Rutherford', 'George Reeves', 'Fred Crane', 'Hattie McDaniel', 'Oscar Polk', 'Butterfly McQueen', 'Victor Jory', 'Everett Brown', 'Howard Hickman', 'Alicia Rhett', 'Leslie Howard', 'Olivia de Havilland', 'Rand Brooks'], 'Public': '1939', 'Pays d’origine': 'United States'}


Quel est le film le plus long ?


Quels sont les 5 films les mieux notés ?

In [9]:
cursor = collection.find({},{'Titre original':1,'Score sur 10':1,'_id':0}).sort("Score sur 10", -1).limit(5)
for i in cursor:
    print(i)

{'Titre original': 'The Shawshank Redemption', 'Score sur 10': '9.3'}
{'Titre original': 'The Godfather', 'Score sur 10': '9.2'}
{'Titre original': "Schindler's List", 'Score sur 10': '9.0'}
{'Titre original': 'The Godfather Part II', 'Score sur 10': '9.0'}
{'Titre original': 'The Lord of the Rings: The Return of the King', 'Score sur 10': '9.0'}


Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [10]:
count = collection.count_documents({"Acteurs": {"$in": ["Tom Cruise"]}})


1

In [11]:
count = collection.count_documents({"Acteurs": {"$in": ["Morgan Freeman"]}})
count

7

Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [21]:
pipeline = [
    {"$match":{"Genre":"Drama"}},
    {'$sort':{"Score sur 10": -1}},
    {'$limit': 3},
    {'$project':{'Titre original':1,'Score sur 10':1,'_id':0}}
]
result = collection.aggregate(pipeline)
for i in result:
    print(i)

{'Titre original': 'The Shawshank Redemption', 'Score sur 10': '9.3'}
{'Titre original': 'Fight Club', 'Score sur 10': '8.8'}
{'Titre original': 'Forrest Gump', 'Score sur 10': '8.8'}


In [22]:
pipeline = [
    {"$match":{"Genre":"Comedy"}},
    {'$sort':{"Score sur 10": -1}},
    {'$limit': 3},
    {'$project':{'Titre original':1,'Score sur 10':1,'_id':0}}
]
result = collection.aggregate(pipeline)
for i in result:
    print(i)

{'Titre original': 'Life Is Beautiful', 'Score sur 10': '8.6'}
{'Titre original': 'City Lights', 'Score sur 10': '8.5'}
{'Titre original': 'Modern Times', 'Score sur 10': '8.5'}


In [23]:
pipeline = [
    {"$match":{"Genre":"Horror"}},
    {'$sort':{"Score sur 10": -1}},
    {'$limit': 3},
    {'$project':{'Titre original':1,'Score sur 10':1,'_id':0}}
]
result = collection.aggregate(pipeline)
for i in result:
    print(i)

{'Titre original': 'Psycho', 'Score sur 10': '8.5'}
{'Titre original': 'Alien', 'Score sur 10': '8.5'}
{'Titre original': 'The Thing', 'Score sur 10': '8.2'}


Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [87]:
pipeline = [
    {'$sort':{"Score sur 10": -1}},
    {'$limit': 100},
    {'$match': {'Pays d’origine': {'$in': ['France', 'United States']}}},
    {'$group': {
        '_id': '$Pays d’origine',
        'num_movies': {'$sum': 1}}}
    
]
results = collection.aggregate(pipeline)

In [88]:
for result in results:
    print(f"Les film d'origine {result['_id']} representent {result['num_movies']} % des 100 meilleur films.")

Les film d'origine United States representent 62 % des 100 meilleur films.
Les film d'origine France representent 4 % des 100 meilleur films.


Quel est la durée moyenne d’un film en fonction du genre ?

In [63]:
pipeline = [
    
    {'$group': {
        '_id': '$Genre',
        'average_duration': {'$avg': '$Duree'}
    }}
]

In [64]:
results = collection.aggregate(pipeline)


In [65]:
for result in results:
    print(f"Le genre {result['_id']} a une durée moyenne de {result['average_duration']} minutes.")

Le genre Comedy a une durée moyenne de 111.43478260869566 minutes.
Le genre Action a une durée moyenne de 139.41860465116278 minutes.
Le genre Biography a une durée moyenne de 142.77272727272728 minutes.
Le genre Mystery a une durée moyenne de 122.75 minutes.
Le genre Drama a une durée moyenne de 129.4848484848485 minutes.
Le genre Western a une durée moyenne de 148.5 minutes.
Le genre Film-Noir a une durée moyenne de 93.0 minutes.
Le genre Adventure a une durée moyenne de 142.38095238095238 minutes.
Le genre Crime a une durée moyenne de 132.6 minutes.
Le genre Animation a une durée moyenne de 100.95652173913044 minutes.
Le genre Horror a une durée moyenne de 114.25 minutes.
